# Подготовительные действия

In [1]:
from utils import *

In [2]:
t = Timer()

# Загрузка рецензий

In [3]:
import pandas as pd

In [4]:
t.start()

reviews_df = pd.read_csv('reviews.csv', sep='\t')
print(reviews_df.shape)

t.stop()

(256509, 2)
Завершено за 00:00:17.254


Распределение классов рецензий

In [5]:
all_num = len(reviews_df)
pos_num = len(reviews_df[reviews_df.mark == 'POSITIVE'])
neg_num = len(reviews_df[reviews_df.mark == 'NEGATIVE'])
neu_num = len(reviews_df[reviews_df.mark == 'NEUTRAL'])

In [6]:
nums = [pos_num, neg_num, neu_num]
data = {'класс': ['Позитивные', 'Негативные', 'Нейтральные'], 
        'число': nums, 
        'доля': [f'{round(num / all_num * 100)}%' for num in nums]}

pd.DataFrame(data).style.hide_index()

класс,число,доля
Позитивные,189079,74%
Негативные,25463,10%
Нейтральные,41967,16%


# Токенизация

In [7]:
import re

In [8]:
TOKEN_RE = re.compile(r'(?:[\w\d]+\-[\w\d]+)|[\w\d]+')

In [9]:
t.start()

tokenized = [TOKEN_RE.findall(review.lower()) 
             for review 
             in reviews_df.review.to_list()]

assert len(tokenized) == all_num

t.stop()

Завершено за 00:01:20.162


# Обучение Word2Vec

In [10]:
import gensim
import multiprocessing

Каждый токен (слово) кодируется вектором длиной EMBEDDING_SIZE

In [11]:
EMBEDDING_SIZE = 32

In [12]:
t.start()

model = gensim.models.Word2Vec(sentences=tokenized, 
                               vector_size=EMBEDDING_SIZE,
                               window=8,
                               min_count=3,
                               workers=multiprocessing.cpu_count())

t.stop()

Завершено за 00:06:23.749


Размер словаря Word2Vec

In [13]:
print(f'Всего уникальных слов: {len(model.wv)}')

Всего уникальных слов: 388740


Проверка работы модели (поиск ближайшего вектора к слову)

In [14]:
model.wv.most_similar('герой')

[('персонаж', 0.8394551277160645),
 ('антагонист', 0.8173109889030457),
 ('злодей', 0.8028539419174194),
 ('зловред', 0.7872943878173828),
 ('протагонист', 0.7781420350074768),
 ('герой-подросток', 0.7620320916175842),
 ('антогонист', 0.756612241268158),
 ('оппонент', 0.7480689287185669),
 ('подозреваемый', 0.7476199865341187),
 ('плохиш', 0.7464181780815125)]

# Преобразование модели Word2Vec в тензор

In [15]:
import torch
import torch.nn as nn

In [16]:
null_vector = torch.zeros(1, EMBEDDING_SIZE, dtype=torch.float)
model_vectors = torch.tensor(model.wv.vectors, dtype=torch.float)
emb_vectors = torch.cat([null_vector, model_vectors])

assert emb_vectors.size()[0] == len(model.wv) + 1

Перевод всех слов в словарь, для нулевого слова используется символ "-"

In [17]:
words = {'-': 0, **{word: i+1 for word, i in model.wv.key_to_index.items()}}

assert len(words) == emb_vectors.size()[0]

# Сохраненние тензора с эмбеддингами и словаря

In [18]:
import pickle

In [19]:
with open('token_dict.pkl', 'wb') as file:
    pickle.dump(words, file)
    
torch.save(emb_vectors, 'embedding.pt')